In [4]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,57.79,76,100,13.44,overcast clouds
1,1,Port Alfred,ZA,-33.5906,26.8910,62.44,82,89,6.44,overcast clouds
2,2,Kapaa,US,22.0752,-159.3190,82.38,72,20,16.11,few clouds
3,3,Jamestown,US,42.0970,-79.2353,77.05,44,0,11.50,clear sky
4,4,Bethel,US,41.3712,-73.4140,88.81,44,0,8.05,clear sky


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Kapaa,US,22.0752,-159.3190,82.38,72,20,16.11,few clouds
3,3,Jamestown,US,42.0970,-79.2353,77.05,44,0,11.50,clear sky
10,10,Ponta Do Sol,PT,32.6667,-17.1000,72.72,73,61,3.22,broken clouds
11,11,Pacific Grove,US,36.6177,-121.9166,84.38,73,0,13.80,clear sky
15,15,Atuona,PF,-9.8000,-139.0333,77.18,68,40,25.52,scattered clouds
16,16,Hithadhoo,MV,-0.6000,73.0833,81.57,75,5,6.98,moderate rain
19,19,Kutum,SD,14.2000,24.6667,75.76,62,88,5.82,overcast clouds
22,22,Nouadhibou,MR,20.9310,-17.0347,73.38,78,4,23.02,clear sky
23,23,Itapage,BR,-3.6867,-39.5861,76.98,62,87,7.40,overcast clouds
24,24,Aloleng,PH,16.1307,119.7824,80.10,81,100,2.62,overcast clouds


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kapaa,US,82.38,few clouds,22.0752,-159.3190,
3,Jamestown,US,77.05,clear sky,42.0970,-79.2353,
10,Ponta Do Sol,PT,72.72,broken clouds,32.6667,-17.1000,
11,Pacific Grove,US,84.38,clear sky,36.6177,-121.9166,
15,Atuona,PF,77.18,scattered clouds,-9.8000,-139.0333,
16,Hithadhoo,MV,81.57,moderate rain,-0.6000,73.0833,
19,Kutum,SD,75.76,overcast clouds,14.2000,24.6667,
22,Nouadhibou,MR,73.38,clear sky,20.9310,-17.0347,
23,Itapage,BR,76.98,overcast clouds,-3.6867,-39.5861,
24,Aloleng,PH,80.10,overcast clouds,16.1307,119.7824,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kapaa,US,82.38,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort Hotel
3,Jamestown,US,77.05,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
10,Ponta Do Sol,PT,72.72,broken clouds,32.6667,-17.1000,Hotel do Campo
11,Pacific Grove,US,84.38,clear sky,36.6177,-121.9166,Lovers Point Inn
15,Atuona,PF,77.18,scattered clouds,-9.8000,-139.0333,Villa Enata


In [16]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))